In [107]:
import pandas as pd
from dotenv import load_dotenv
import os
from google.oauth2 import service_account
from googleapiclient.discovery import build

load_dotenv()

gtm_id = os.environ["GTM_ACCOUNT_ID"]

In [126]:
def get_credentials():
    SERVICE_ACCOUNT_FILE = './xlsx/gtm-api-409502-6f8362f190d7.json'

    credentials_container = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE,
        scopes=["https://www.googleapis.com/auth/tagmanager.edit.containers"]
    )

    return credentials_container

def get_service(credentials):
    service = build('tagmanager', 'v2', credentials = credentials)

    return service

def get_container(service, gtm_id):
    parent = 'accounts/' + gtm_id
    containers = service.accounts().containers().list(
        parent=parent,
    ).execute()

    return containers

def get_workspaces(service, container_id, gtm_id):
    parent='accounts/'+gtm_id+'/containers/'+container_id
    workspace = service.accounts().containers().workspaces().list(
        parent=parent
    ).execute()

    return workspace['workspace']

def get_variables(service, container_id, workspace_id, gtm_id):
    parent='accounts/'+gtm_id+'/containers/'+container_id+'/workspaces/'+workspace_id
    containers = service.accounts().containers().workspaces().variables().list(
        parent=parent
    ).execute()

    return containers
    

In [109]:
credentials = get_credentials()
service = get_service(credentials)
container = get_container(services['container'], gtm_id)['container']
container_id = container[1]['containerId']

workspace = get_workspaces(services['container'], container_id, gtm_id)
workspace_id = workspace[0]['workspaceId']

variables = get_variables(services['container'], container_id, workspace_id, gtm_id)['variable']


In [106]:
{
    'path': 'accounts/6003640738/containers/90556056/workspaces/45/variables/283',
    'accountId': '6003640738',
    'containerId': '90556056',
    'workspaceId': '45',
    'variableId': '283',
    'name': 'subscribe',
    'type': 'jsm',
    'parameter': [{
        'type': 'template',
        'key': 'javascript',
        'value': 'function() {\n  return {{Click Element}}.getAttribute("data-gtm-click");\n}'
    }],
    'fingerprint': '1698633207560',
    'tagManagerUrl': 'https://tagmanager.google.com/#/container/accounts/6003640738/containers/90556056/workspaces/45/variables/283?apiLink=variable',
    'formatValue': {}
}

{'path': 'accounts/6003640738/containers/90556056/workspaces/45/variables/283',
 'accountId': '6003640738',
 'containerId': '90556056',
 'workspaceId': '45',
 'variableId': '283',
 'name': 'subscribe',
 'type': 'jsm',
 'parameter': [{'type': 'template',
   'key': 'javascript',
   'value': 'function() {\n  return {{Click Element}}.getAttribute("data-gtm-click");\n}'}],
 'fingerprint': '1698633207560',
 'tagManagerUrl': 'https://tagmanager.google.com/#/container/accounts/6003640738/containers/90556056/workspaces/45/variables/283?apiLink=variable',
 'formatValue': {}}

In [114]:
for key, value in variables[0].items():
    print(key, value)

path accounts/6003640738/containers/90556056/workspaces/45/variables/287
accountId 6003640738
containerId 90556056
workspaceId 45
variableId 287
name pageTitle
type j
parameter [{'type': 'template', 'key': 'name', 'value': 'document.title'}]
fingerprint 1698712408514
tagManagerUrl https://tagmanager.google.com/#/container/accounts/6003640738/containers/90556056/workspaces/45/variables/287?apiLink=variable
formatValue {}


In [118]:
def get_sheet_name(excel):
    sheet_names = excel.sheet_names
    return sheet_names

def read_sheet(excel_file, sheet_name):
    return pd.read_excel(excel_file, sheet_name=sheet_name)

def read_column_name(sheet):
    return sheet.columns

In [119]:
xls = pd.ExcelFile('./xlsx/library.xlsx')

sheet_names = get_sheet_name(xls)
sheet = read_sheet(xls, sheet_names[0])
column_name = read_column_name(sheet)

In [125]:
for row in sheet.iloc():
    request_body = {
        'name': row[0],
        'type': row[1],
        'value': row[2],
    }
    print(request_body)

{'name': 'GA4 - 측정 ID', 'type': '상수', 'value': 'G-HNNQ7TSLY9'}
{'name': 'DLV - platform', 'type': '데이터 영역 변수', 'value': 'platform'}
{'name': 'DLV - is_member', 'type': '데이터 영역 변수', 'value': 'is_member'}
{'name': 'DLV - library_id', 'type': '데이터 영역 변수', 'value': 'library_id'}
{'name': 'DLV - library', 'type': '데이터 영역 변수', 'value': 'library'}
{'name': 'DLV - library_model', 'type': '데이터 영역 변수', 'value': '데이터 영역 변수'}
{'name': 'DLV - user_id', 'type': '데이터 영역 변수', 'value': 'user_id'}
{'name': 'DLV - method', 'type': '데이터 영역 변수', 'value': 'method'}
{'name': 'DLV - signup_date', 'type': '데이터 영역 변수', 'value': 'signup_date'}
{'name': 'DLV - login_count', 'type': '데이터 영역 변수', 'value': 'login_count'}
{'name': 'DLV - total_borrow', 'type': '데이터 영역 변수', 'value': 'total_borrow'}
{'name': 'DLV - total_return', 'type': '데이터 영역 변수', 'value': 'total_return'}
{'name': 'DLV - total_reservation', 'type': '데이터 영역 변수', 'value': 'total_reservation'}
{'name': 'DLV - recent_login_date', 'type': '데이터 영역 변수', 'v

/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_10670/3078349034.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'name': row[0],
/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_10670/3078349034.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'type': row[1],
/var/folders/xq/qf2k1k454qb_jfv62h4qfhn00000gn/T/ipykernel_10670/3078349034.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'value': row[2],
